# Backtesting sistematica

#### Hecho por: Nicolás Florián Ortiz y Juan Esteban Garzón Borda

Se importan las librerias necesarias para realizar el backtesting

In [93]:
# Preámbulo
import pandas as pd
import numpy as np

Para cada año se extrae la base de datos que contiene la información de las empresas pertenecientes

In [95]:
df_2015 = pd.read_excel('Datos estrategia sistematica.xlsx', sheet_name='2015', header=[0, 1], index_col=0)
df_2016 = pd.read_excel('Datos estrategia sistematica.xlsx', sheet_name='2016', header=[0, 1], index_col=0)
df_2017 = pd.read_excel('Datos estrategia sistematica.xlsx', sheet_name='2017', header=[0, 1], index_col=0)
df_2018 = pd.read_excel('Datos estrategia sistematica.xlsx', sheet_name='2018', header=[0, 1], index_col=0)
df_2019 = pd.read_excel('Datos estrategia sistematica.xlsx', sheet_name='2019', header=[0, 1], index_col=0)
df_2020 = pd.read_excel('Datos estrategia sistematica.xlsx', sheet_name='2020', header=[0, 1], index_col=0)
df_2021 = pd.read_excel('PRUEBA.xlsx', header=[0, 1], index_col=0)
df_2022 = pd.read_excel('Datos estrategia sistematica.xlsx', sheet_name='2022', header=[0, 1], index_col=0)
df_2023 = pd.read_excel('Datos estrategia sistematica.xlsx', sheet_name='2023', header=[0, 1], index_col=0)

In [108]:
df_2021 = pd.read_excel('PRUEBA.xlsx', header=[0, 1], index_col=0)
df_2019 = pd.read_excel('PRUEBA_2019.xlsx', header=[0, 1], index_col=0)


Se define la función que realiza el backtesting para la base de datos

In [109]:
def Backtesting(capital_inicial, ventana_inicial, df):


    datos_por_empresa = {}

    # Iterar a través de las columnas del DataFrame original
    for col_idx in range(0, len(df.columns.levels[0]), 4):
        # Obtener el nombre de la empresa del índice de la columna
        nombre_empresa_original = df.columns[col_idx][0]
    
        # Extraer el segundo nombre de la empresa (el que está después de los dos puntos)
        if ':' in nombre_empresa_original:
            partes_nombre = nombre_empresa_original.split(':')
            nombre_empresa_nuevo = ':'.join(partes_nombre[1:]).strip()
        else:
            nombre_empresa_nuevo = nombre_empresa_original  # Si no hay dos puntos, mantener el nombre original
    
        # Seleccionar las columnas correspondientes para la empresa
        datos_empresa = df.iloc[:, col_idx:col_idx+4]
    
        # Convertir a números (asumiendo que estas columnas deben ser numéricas)
        datos_empresa = datos_empresa.apply(pd.to_numeric, errors='coerce')
    
        # Verificar si hay valores NA en algún lugar de los datos de la empresa
        if not datos_empresa.isnull().values.any() and 'PX_LAST' in datos_empresa.columns.get_level_values(1):
            # Almacenar en el diccionario con el nuevo nombre como clave
            datos_por_empresa[nombre_empresa_nuevo] = datos_empresa

 
    
    # Ahora, vamos a actualizar el diccionario con los nuevos nombres
    for nombre_original, datos_empresa in datos_por_empresa.items():
        # Extraer el segundo nombre de la empresa (el que está después de los dos puntos)
        if ':' in nombre_original:
            nombre_nuevo = nombre_original.split(':')[1].strip()
            # Actualizar el diccionario con el nuevo nombre como clave
            datos_por_empresa[nombre_nuevo] = datos_por_empresa.pop(nombre_original)


    # Crear un diccionario para almacenar los rendimientos por empresa
    rendimientos_por_empresa = {}
    
    # Calcular el rendimiento para cada empresa
    for empresa, datos_empresa in datos_por_empresa.items():
        # Obtener el precio en la posición 61 y el precio en la primera posición
        precio_posicion_61 = datos_empresa.iloc[ventana_inicial, 3]  # Tomar la cuarta columna sin importar el nombre
        precio_primera_posicion = datos_empresa.iloc[(ventana_inicial-60), 3]  # Tomar la cuarta columna sin importar el nombre
    
        # Calcular el rendimiento
        rendimiento = ((precio_posicion_61 / precio_primera_posicion) - 1) * 100
    
        # Almacenar el rendimiento en el diccionario
        rendimientos_por_empresa[empresa] = rendimiento
    
    # Crear un DataFrame a partir del diccionario de rendimientos
    df_rendimientos = pd.DataFrame(list(rendimientos_por_empresa.items()), columns=['Empresa', 'Rendimiento'])
    
    # Ordenar el DataFrame por rendimiento de mayor a menor
    df_rendimientos = df_rendimientos.sort_values(by='Rendimiento', ascending=False)
    
    # Tomar las primeras 15 posiciones como "Largo"
    df_largo = df_rendimientos.head(15)
    
    # Tomar las últimas 5 posiciones como "Corto"
    df_corto = df_rendimientos.tail(5)
    
    # Crear una nueva matriz para df_largo
    matriz_disminucion_pe_largo = pd.DataFrame(index=df_largo['Empresa'], columns=['Disminucion_PE'])

    # Iterar sobre las empresas en df_largo
    for idx, empresa in enumerate(df_largo['Empresa']):
        # Acceder a 'PE_RATIO' en el DataFrame de la empresa
        columna_pe_ratio = (empresa, 'PE_')
        
        # Obtener el PE_RATIO de la observación 61 en df_largo
        PE_RATIO_observacion_61_largo = datos_por_empresa[empresa][columna_pe_ratio].iloc[ventana_inicial]
    
        # Obtener el PE_RATIO de la primera observación para cada empresa en df_largo
        PE_RATIO_primera_observacion_largo = datos_por_empresa[empresa][columna_pe_ratio].iloc[(ventana_inicial-60)]
    
        # Marcar con 1 si hay disminución en PE_RATIO, de lo contrario, con 0
        matriz_disminucion_pe_largo.loc[empresa, 'Disminucion_PE'] = int(PE_RATIO_primera_observacion_largo > PE_RATIO_observacion_61_largo)
    
    # Para las cortas
    matriz_disminucion_pe_corto = pd.DataFrame(index=df_corto['Empresa'], columns=['Aumento_PE'])
    
    # Iterar sobre las empresas en df_largo
    for idx, empresa in enumerate(df_corto['Empresa']):
        # Acceder a 'PE_RATIO' en el DataFrame de la empresa
        columna_pe_ratio_c = (empresa, 'PE_')
        
        # Obtener el PE_RATIO de la observación 61 en df_largo
        PE_RATIO_observacion_61_corto = datos_por_empresa[empresa][columna_pe_ratio_c].iloc[ventana_inicial]
    
        # Obtener el PE_RATIO de la primera observación para cada empresa en df_largo
        PE_RATIO_primera_observacion_corto = datos_por_empresa[empresa][columna_pe_ratio_c].iloc[(ventana_inicial-1)]
    
        # Marcar con 1 si hay disminución en PE_RATIO, de lo contrario, con 0
        matriz_disminucion_pe_corto.loc[empresa, 'Aumento_PE'] = int(PE_RATIO_primera_observacion_largo < PE_RATIO_observacion_61_corto)
    
    # Crear matriz_MACO_largos
    matriz_MACO_largos = pd.DataFrame(index=df_largo['Empresa'], columns=['MACO'])
    
    # Iterar sobre las empresas en df_largo
    for empresa in df_largo['Empresa']:
        if empresa in datos_por_empresa:
            # Obtener los precios de cierre para la empresa
            precios_cierre = datos_por_empresa[empresa][empresa, 'PX_LAST']
    
            CP = 5
            LP = 30
            promedio_corto_plazo = precios_cierre.iloc[(ventana_inicial-CP):ventana_inicial].mean()
            promedio_largo_plazo = precios_cierre.iloc[(ventana_inicial-LP):ventana_inicial].mean()
    
            # Marcar con 1 si el promedio del corto plazo es mayor que el del largo plazo, de lo contrario, con 0
            matriz_MACO_largos.loc[empresa, 'MACO'] = int(promedio_corto_plazo > promedio_largo_plazo)
    
    # Para los cortos
    matriz_MACO_cortos = pd.DataFrame(index=df_corto['Empresa'], columns=['MACO'])
    
    for empresa in df_corto['Empresa']:
        if empresa in datos_por_empresa:
            pruebitas = datos_por_empresa[empresa].reset_index(drop = True)
            precios_cierre_c = datos_por_empresa[empresa][empresa].iloc[3]
            CP = 5
            LP = 30
            promedio_corto_plazo_c = precios_cierre_c.iloc[(ventana_inicial-CP):ventana_inicial].mean()
            promedio_largo_plazo_c = precios_cierre_c.iloc[(ventana_inicial-LP):ventana_inicial].mean()
    
            # Marcar con 1 si el promedio del corto plazo es mayor que el del largo plazo, de lo contrario, con 0
            matriz_MACO_cortos.loc[empresa, 'MACO'] = int(promedio_corto_plazo_c < promedio_largo_plazo_c)

    # Crear una nueva matriz para posiciones largas
    matriz_largo_ewma = pd.DataFrame(index=df_largo['Empresa'], columns=['Posicion'])
    
    # Factor de suavización para EWMA
    alpha = 2/(14+1)
    
    # Posición en la que estamos revisando la señal de compra
    posicion_actual = ventana_inicial
    
    # Aplicar EWMA a las empresas en df_largo y asignar señales de compra
    for empresa in df_largo['Empresa']:
        if empresa in datos_por_empresa:
            precios_cierre = datos_por_empresa[empresa].loc[:, (slice(None), 'PX_LAST')]  # Seleccionar la columna 'PX_LAST'
            ewma = precios_cierre.ewm(alpha=alpha, adjust=False).mean()
            
            # Comparar el valor en la posición actual con el valor de EWMA en la posición actual
            if precios_cierre.iloc[posicion_actual].max() > ewma.iloc[posicion_actual].max():
                matriz_largo_ewma.loc[empresa, 'Posicion'] = 1
            else:
                matriz_largo_ewma.loc[empresa, 'Posicion'] = 0
    
    matriz_corto_ewma = pd.DataFrame(index=df_corto['Empresa'], columns=['Posicion'])

    # Aplicar EWMA a las empresas en df_corto y asignar señales de venta
    for empresa in df_corto['Empresa']:
        if empresa in datos_por_empresa:
            precios_cierre = datos_por_empresa[empresa].loc[:, (slice(None), 'PX_LAST')]  # Seleccionar la columna 'PX_LAST'
            ewma = precios_cierre.ewm(alpha=alpha, adjust=False).mean()
            
            # Comparar el valor en la posición actual con el valor de EWMA en la posición actual
            if precios_cierre.iloc[posicion_actual].max() < ewma.iloc[posicion_actual].max():
                matriz_corto_ewma.loc[empresa, 'Posicion'] = 1
            else:
                matriz_corto_ewma.loc[empresa, 'Posicion'] = 0



    # Crear matrices para señales fuertes y débiles en largos y cortos
    matriz_fuerte_largo = pd.DataFrame(index=df_largo['Empresa'], columns=['Senal_Fuerte'])
    matriz_debil_largo = pd.DataFrame(index=df_largo['Empresa'], columns=['Senal_Debil'])
    matriz_fuerte_corto = pd.DataFrame(index=df_corto['Empresa'], columns=['Senal_Fuerte'])
    matriz_debil_corto = pd.DataFrame(index=df_corto['Empresa'], columns=['Senal_Debil'])

    # Iterar sobre las empresas en df_largo
    for empresa in df_largo['Empresa']:
        if empresa in datos_por_empresa:
            # Verificar condiciones para señal fuerte y débil en largos
            condicion_fuerte_largo = matriz_disminucion_pe_largo.loc[empresa, 'Disminucion_PE'] == 1 and matriz_MACO_largos.loc[empresa, 'MACO'] == 1
            condicion_debil_largo = matriz_disminucion_pe_largo.loc[empresa, 'Disminucion_PE'] == 1 or matriz_MACO_largos.loc[empresa, 'MACO'] == 1
    
            # Marcar con 1 si cumple la condición de señal fuerte, de lo contrario, con 0
            matriz_fuerte_largo.loc[empresa, 'Senal_Fuerte'] = int(condicion_fuerte_largo)
            # Marcar con 1 si cumple la condición de señal débil, de lo contrario, con 0
            matriz_debil_largo.loc[empresa, 'Senal_Debil'] = int(condicion_debil_largo)

        # Iterar sobre las empresas en df_corto
    for empresa in df_corto['Empresa']:
        if empresa in datos_por_empresa:
            # Verificar condiciones para señal fuerte y débil en cortos
            condicion_fuerte_corto = matriz_disminucion_pe_corto.loc[empresa, 'Aumento_PE'] == 1 and matriz_MACO_cortos.loc[empresa, 'MACO'] == 1
            condicion_debil_corto = matriz_disminucion_pe_corto.loc[empresa, 'Aumento_PE'] == 1 or matriz_MACO_cortos.loc[empresa, 'MACO'] == 1
    
            # Marcar con 1 si cumple la condición de señal fuerte, de lo contrario, con 0
            matriz_fuerte_corto.loc[empresa, 'Senal_Fuerte'] = int(condicion_fuerte_corto)
            # Marcar con 1 si cumple la condición de señal débil, de lo contrario, con 0
            matriz_debil_corto.loc[empresa, 'Senal_Debil'] = int(condicion_debil_corto)

    # Crear matrices para señales fuertes y débiles en largos y cortos
    matriz_fuerte_largo = pd.DataFrame(index=df_largo['Empresa'], columns=['Senal_Fuerte'])
    matriz_debil_largo = pd.DataFrame(index=df_largo['Empresa'], columns=['Senal_Debil'])
    matriz_fuerte_corto = pd.DataFrame(index=df_corto['Empresa'], columns=['Senal_Fuerte'])
    matriz_debil_corto = pd.DataFrame(index=df_corto['Empresa'], columns=['Senal_Debil'])
    
    # Iterar sobre las empresas en df_largo
    for empresa in df_largo['Empresa']:
        if empresa in datos_por_empresa:
            # Verificar condiciones para señal fuerte y débil en largos
            condicion_fuerte_largo = matriz_disminucion_pe_largo.loc[empresa, 'Disminucion_PE'] == 1 and matriz_MACO_largos.loc[empresa, 'MACO'] == 1
            condicion_debil_largo = matriz_disminucion_pe_largo.loc[empresa, 'Disminucion_PE'] == 1 or matriz_MACO_largos.loc[empresa, 'MACO'] == 1
    
            # Si cumple todas las condiciones, asignar a señal fuerte
            if condicion_fuerte_largo:
                matriz_fuerte_largo.loc[empresa, 'Senal_Fuerte'] = 1
            # Si cumple al menos una condición, asignar a señal débil
            elif condicion_debil_largo:
                matriz_debil_largo.loc[empresa, 'Senal_Debil'] = 1
    
    # Iterar sobre las empresas en df_corto
    for empresa in df_corto['Empresa']:
        if empresa in datos_por_empresa:
            # Verificar condicione para señal fuerte y débil en cortos
            condicion_fuerte_corto = matriz_disminucion_pe_corto.loc[empresa, 'Aumento_PE'] == 1 and matriz_MACO_cortos.loc[empresa, 'MACO'] == 1
            condicion_debil_corto = matriz_disminucion_pe_corto.loc[empresa, 'Aumento_PE'] == 1 or matriz_MACO_cortos.loc[empresa, 'MACO'] == 1
    
            # Si cumple todas las condiciones, asignar a señal fuerte
            if condicion_fuerte_corto:
                matriz_fuerte_corto.loc[empresa, 'Senal_Fuerte'] = 1
            # Si cumple al menos una condición, asignar a señal débil
            elif condicion_debil_corto:
                matriz_debil_corto.loc[empresa, 'Senal_Debil'] = 1
    
    
    # Combina las empresas de señal débil para largos y cortos en una sola matriz
    matriz_debil_total = pd.concat([matriz_debil_largo, matriz_debil_corto])

    # Asigna pesos usando la metodología de "volatility parity"
    # Calcula la volatilidad de cada empresa
    volatilidades_debil = {}
    for empresa in matriz_debil_total.index:
        if empresa in datos_por_empresa:
            # Obtén la última columna de la matriz de datos_por_empresa para evitar el KeyError
            columna_px_last = datos_por_empresa[empresa].columns[-1]
            
            # Calcula el rendimiento logarítmico diario directamente desde la última columna
            rendimientos = np.log(datos_por_empresa[empresa][columna_px_last] / datos_por_empresa[empresa][columna_px_last].shift(1))
            
            # Calcula la volatilidad como la desviación estándar del rendimiento logarítmico
            volatilidad = rendimientos.std()

            # Almacena la volatilidad en el diccionario
            volatilidades_debil[empresa] = volatilidad

    # Normaliza las volatilidades
    total_volatilidad_debil = sum(volatilidades_debil.values())
    volatilidades_normalizadas_debil = {empresa: vol / total_volatilidad_debil for empresa, vol in volatilidades_debil.items()}
    
    # Crea un DataFrame para almacenar los nombres de las empresas y los pesos asignados
    df_pesos_debil = pd.DataFrame(list(volatilidades_normalizadas_debil.items()), columns=['Empresa', 'Peso'])

    # Definir las matrices de señales fuertes para largos y cortos
    matriz_fuerte_total = pd.concat([matriz_fuerte_largo.dropna(), matriz_fuerte_corto.dropna()])
    
    # Filtrar empresas con señales fuertes
    empresas_fuertes = matriz_fuerte_total.index
    
    # Crear un DataFrame para almacenar las volatilidades y los precios de cierre de las empresas fuertes
    df_vol_precios = pd.DataFrame(index=empresas_fuertes, columns=['Volatilidad', 'Precios'])
    
    # Calcular volatilidad y asignar precios
    for empresa in empresas_fuertes:
        if empresa in datos_por_empresa:
            # Obtener la columna de precios (cuarta columna)
            precios_cierre = datos_por_empresa[empresa].iloc[0:ventana_inicial, 3]
    
            # Calcular rendimientos diarios negativos
            rendimientos_negativos = np.log(precios_cierre / precios_cierre.shift(1))
            rendimientos_negativos = rendimientos_negativos[rendimientos_negativos < 0]
    
            # Calcular volatilidad como desviación estándar de rendimientos negativos
            volatilidad = rendimientos_negativos.std()
    
            # Almacenar en el DataFrame
            df_vol_precios.loc[empresa, 'Volatilidad'] = volatilidad
            df_vol_precios.loc[empresa, 'Precios'] = precios_cierre.tolist()  # Almacenar la serie completa

    # Ordenar empresas por volatilidad de mayor a menor
    df_vol_precios = df_vol_precios.sort_values(by='Volatilidad', ascending=False)
    
    # Asignar valores objetivos según posición en la lista ordenada
    n_empresas = len(df_vol_precios)
    df_vol_precios['Valor_Objetivo'] = np.arange(0.05, 0.05 * (n_empresas + 1), 0.05)[:n_empresas]

    
    # Calcular valor operativo para cada empresa
    df_vol_precios['Valor_Operativo'] = df_vol_precios['Valor_Objetivo'] / df_vol_precios['Volatilidad']
    
    # Calcular total valor operativo
    total_valor_operativo = df_vol_precios['Valor_Operativo'].sum()
    
    # Calcular ponderaciones
    df_vol_precios['Ponderacion'] = df_vol_precios['Valor_Operativo'] / total_valor_operativo
    
    # Crear DataFrame final con nombre de la empresa y ponderación
    df_ponderaciones = df_vol_precios[['Ponderacion']].reset_index()

    # Calcular el capital destinado para las señales fuertes (35% del capital inicial)
    capital_fuerte = capital_inicial * 0.35
    
    # Calcular el capital destinado para las señales débiles (65% del capital inicial)
    capital_debil = capital_inicial * 0.65
    
    # Crear una columna en df_ponderaciones para almacenar el capital invertido
    df_ponderaciones['Capital_Invertido'] = df_ponderaciones['Ponderacion'] * capital_fuerte
    
    # Crear una columna en df_pesos_debil para almacenar el capital invertido
    df_pesos_debil['Capital_Invertido'] = df_pesos_debil['Peso'] * capital_debil
    
    # Crear una nueva columna en df_ponderaciones para almacenar el rendimiento
    df_ponderaciones['Rendimiento'] = 0.0
    
    # Crear una nueva columna en df_pesos_debil para almacenar el rendimiento
    df_pesos_debil['Rendimiento'] = 0.0

    # Iterar sobre las empresas en df_ponderaciones
    for idx, row in df_ponderaciones.iterrows():
        empresa = row['Empresa']
        capital_invertido = row['Capital_Invertido']
        
        # Verificar si la empresa es larga en la matriz de señales fuertes
        if empresa in matriz_fuerte_largo.index:
            # Calcular el rendimiento como precio final / precio inicial - 1
            rendimiento = (datos_por_empresa[empresa].iloc[(ventana_inicial+5), 3] / datos_por_empresa[empresa].iloc[ventana_inicial, 3]) - 1
        elif empresa in matriz_fuerte_corto.index:
            # Calcular el rendimiento como precio inicial / precio final - 1 para cortos
            rendimiento = (datos_por_empresa[empresa].iloc[ventana_inicial, 3] / datos_por_empresa[empresa].iloc[(ventana_inicial + 5), 3]) - 1
        
        # Almacenar el rendimiento en la columna correspondiente
        df_ponderaciones.loc[idx, 'Rendimiento'] = rendimiento


    # Iterar sobre las empresas en df_pesos_debil
    for idx, row in df_pesos_debil.iterrows():
        empresa = row['Empresa']
        capital_invertido = row['Capital_Invertido']
        
        # Verificar si la empresa es larga en la matriz de señales débiles
        if empresa in matriz_debil_largo.index:
            # Calcular el rendimiento como precio final / precio inicial - 1
            rendimiento = (datos_por_empresa[empresa].iloc[(ventana_inicial + 5), 3] / datos_por_empresa[empresa].iloc[ventana_inicial, 3]) - 1
        elif empresa in matriz_debil_corto.index:
            # Calcular el rendimiento como precio inicial / precio final - 1 para cortos
            rendimiento = (datos_por_empresa[empresa].iloc[ventana_inicial, 3] / datos_por_empresa[empresa].iloc[(ventana_inicial + 5), 3]) - 1
        
        # Almacenar el rendimiento en la columna correspondiente
        df_pesos_debil.loc[idx, 'Rendimiento'] = rendimiento

    # Sumar los rendimientos para obtener el rendimiento total
    rendimiento_total_fuerte = df_ponderaciones['Rendimiento'].sum()
    rendimiento_total_debil = df_pesos_debil['Rendimiento'].sum()
    
    # Calcular el dinero final
    dinero_final_fuerte = capital_fuerte * (1 + rendimiento_total_fuerte)
    dinero_final_debil = capital_debil * (1 + rendimiento_total_debil)
    
    # Calcular el dinero total final
    dinero_total_final = dinero_final_fuerte + dinero_final_debil
    
    # Calcular la diferencia con el capital inicial
    diferencia = dinero_total_final - capital_inicial

    
    # Mostrar los resultados
    print("Dinero inicial: $", capital_inicial)
    print("Dinero Final Fuerte: $", dinero_final_fuerte)
    print("Dinero Final Débil: $", dinero_final_debil)
    print("Dinero Total Final: $", dinero_total_final)
    print("Diferencia con el Capital Inicial: $", diferencia)
    print("RDTO:" , (dinero_total_final/capital_inicial -1))

    

Para el 2015 

In [32]:
for i in range(60, 260, 5):
    Backtesting(1000000, i , df_2015)

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 377999.99999999994
Dinero Final Débil: $ 966344.9739268114
Dinero Total Final: $ 1344344.9739268112
Diferencia con el Capital Inicial: $ 344344.97392681125
RDTO: 0.3443449739268112
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 768413.5666286141
Dinero Total Final: $ 1118413.566628614
Diferencia con el Capital Inicial: $ 118413.56662861397
RDTO: 0.11841356662861391
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ -89077.35197877488
Dinero Total Final: $ 260922.64802122512
Diferencia con el Capital Inicial: $ -739077.3519787749
RDTO: -0.7390773519787749
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 742681.3195038651
Dinero Total Final: $ 1092681.3195038652
Diferencia con el Capital Inicial: $ 92681.3195038652
RDTO: 0.09268131950386516
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 440084.9714684849
Dinero Tota

Para el 2016

In [112]:
for i in range(60, 255, 5):
    Backtesting(1000000, i , df_2016)

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 707235.7390465525
Dinero Total Final: $ 1057235.7390465525
Diferencia con el Capital Inicial: $ 57235.73904655245
RDTO: 0.05723573904655255
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 354751.1312217195
Dinero Final Débil: $ 816077.5755435039
Dinero Total Final: $ 1170828.7067652233
Diferencia con el Capital Inicial: $ 170828.7067652233
RDTO: 0.17082870676522321
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 382812.5
Dinero Final Débil: $ 876451.7613474169
Dinero Total Final: $ 1259264.2613474168
Diferencia con el Capital Inicial: $ 259264.2613474168
RDTO: 0.25926426134741676
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 373142.5004458712
Dinero Final Débil: $ 669879.3066182988
Dinero Total Final: $ 1043021.80706417
Diferencia con el Capital Inicial: $ 43021.80706417002
RDTO: 0.043021807064169915
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 361612.13706962444
Dinero Final Débil: $ 724921.519899

Para el 2017

In [116]:
for i in range(60, 255, 5):
    Backtesting(1000000, i , df_2017)

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 758159.0568720237
Dinero Total Final: $ 1108159.0568720237
Diferencia con el Capital Inicial: $ 108159.05687202374
RDTO: 0.10815905687202365
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 318701.79907169146
Dinero Final Débil: $ 813023.6817255478
Dinero Total Final: $ 1131725.480797239
Diferencia con el Capital Inicial: $ 131725.4807972391
RDTO: 0.131725480797239
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350180.97970194294
Dinero Final Débil: $ 588710.1483555547
Dinero Total Final: $ 938891.1280574976
Diferencia con el Capital Inicial: $ -61108.87194250245
RDTO: -0.061108871942502474
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 388186.4316433969
Dinero Final Débil: $ 980885.4207513185
Dinero Total Final: $ 1369071.8523947154
Diferencia con el Capital Inicial: $ 369071.8523947154
RDTO: 0.3690718523947154
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 396029.4748088245
Dinero Final Débil: $ 114

Para el 2018

In [111]:
for i in range(60, 260, 5):
    Backtesting(1000000, i , df_2018)

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 368991.0979228487
Dinero Final Débil: $ 721953.6206348685
Dinero Total Final: $ 1090944.7185577173
Diferencia con el Capital Inicial: $ 90944.71855771728
RDTO: 0.09094471855771724
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 403466.9365206114
Dinero Final Débil: $ 785402.5545935306
Dinero Total Final: $ 1188869.491114142
Diferencia con el Capital Inicial: $ 188869.49111414189
RDTO: 0.18886949111414197
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 355729.53736654803
Dinero Final Débil: $ 459779.63769790804
Dinero Total Final: $ 815509.1750644561
Diferencia con el Capital Inicial: $ -184490.82493554393
RDTO: -0.18449082493554392
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 352892.56198347104
Dinero Final Débil: $ 491301.4316495609
Dinero Total Final: $ 844193.993633032
Diferencia con el Capital Inicial: $ -155806.00636696804
RDTO: -0.15580600636696806
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 362924.8286654795
Dinero Fina

Para el 2019

In [110]:
for i in range(60, 260, 5):
    Backtesting(1000000, i , df_2019)

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 650000.0
Dinero Total Final: $ 1000000.0
Diferencia con el Capital Inicial: $ 0.0
RDTO: 0.0
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 650000.0
Dinero Total Final: $ 1000000.0
Diferencia con el Capital Inicial: $ 0.0
RDTO: 0.0
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 650000.0
Dinero Total Final: $ 1000000.0
Diferencia con el Capital Inicial: $ 0.0
RDTO: 0.0
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 650000.0
Dinero Total Final: $ 1000000.0
Diferencia con el Capital Inicial: $ 0.0
RDTO: 0.0
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 650000.0
Dinero Total Final: $ 1000000.0
Diferencia con el Capital Inicial: $ 0.0
RDTO: 0.0
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 650000.0
Dinero Total Final: $ 1000000.0
Diferencia con el C

Para el 2020

In [113]:
for i in range(60, 260, 5):
    Backtesting(1000000, i , df_2020)    

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 949876.0303154463
Dinero Total Final: $ 1299876.0303154462
Diferencia con el Capital Inicial: $ 299876.0303154462
RDTO: 0.2998760303154462
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 430163.43499747897
Dinero Final Débil: $ 715220.7655808649
Dinero Total Final: $ 1145384.2005783438
Diferencia con el Capital Inicial: $ 145384.20057834382
RDTO: 0.1453842005783439
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 371067.6686134645
Dinero Final Débil: $ 938372.8960005523
Dinero Total Final: $ 1309440.5646140168
Diferencia con el Capital Inicial: $ 309440.5646140168
RDTO: 0.3094405646140168
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 442001.3209887943
Dinero Final Débil: $ 291933.4327198124
Dinero Total Final: $ 733934.7537086067
Diferencia con el Capital Inicial: $ -266065.2462913933
RDTO: -0.2660652462913933
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 376196.6241146825
Dinero Final Débil: $ 47144

Para 2021

In [117]:
for i in range(60, 255, 5):
    Backtesting(1000000, i , df_2021)   

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 384217.7550255962
Dinero Final Débil: $ 1052927.3156162144
Dinero Total Final: $ 1437145.0706418105
Diferencia con el Capital Inicial: $ 437145.07064181054
RDTO: 0.4371450706418105
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 260914.87547744915
Dinero Final Débil: $ 657077.0098934635
Dinero Total Final: $ 917991.8853709126
Diferencia con el Capital Inicial: $ -82008.11462908739
RDTO: -0.08200811462908741
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 359894.4369908872
Dinero Final Débil: $ 547083.8657310506
Dinero Total Final: $ 906978.3027219379
Diferencia con el Capital Inicial: $ -93021.69727806211
RDTO: -0.09302169727806209
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 340490.10824806924
Dinero Final Débil: $ 277766.57479734067
Dinero Total Final: $ 618256.6830454099
Diferencia con el Capital Inicial: $ -381743.31695459015
RDTO: -0.3817433169545902
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 340006.8414451409
Dinero Fin

Para el 2022

In [103]:
for i in range(60, 260, 5):
    Backtesting(1000000, i , df_2022)   

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 910244.0249458773
Dinero Total Final: $ 1260244.0249458773
Diferencia con el Capital Inicial: $ 260244.02494587726
RDTO: 0.2602440249458773
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 347971.67819675856
Dinero Final Débil: $ 929933.3750694722
Dinero Total Final: $ 1277905.0532662307
Diferencia con el Capital Inicial: $ 277905.05326623074
RDTO: 0.27790505326623083
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 395485.5973803868
Dinero Final Débil: $ 1041099.940941532
Dinero Total Final: $ 1436585.5383219188
Diferencia con el Capital Inicial: $ 436585.5383219188
RDTO: 0.4365855383219188
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 283484.04636829504
Dinero Final Débil: $ -284254.8432205077
Dinero Total Final: $ -770.7968522126903
Diferencia con el Capital Inicial: $ -1000770.7968522126
RDTO: -1.0007707968522126
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 417610.32019228785
Dinero Final Débil: 

Para el 2023

In [104]:
for i in range(60, 260, 5):
    Backtesting(1000000, i , df_2023)   

Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 350000.0
Dinero Final Débil: $ 741939.9776548421
Dinero Total Final: $ 1091939.9776548422
Diferencia con el Capital Inicial: $ 91939.9776548422
RDTO: 0.09193997765484219
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 361058.39779926726
Dinero Final Débil: $ 687376.2399563454
Dinero Total Final: $ 1048434.6377556126
Diferencia con el Capital Inicial: $ 48434.63775561261
RDTO: 0.04843463775561263
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 372899.065219182
Dinero Final Débil: $ 976287.3214276835
Dinero Total Final: $ 1349186.3866468654
Diferencia con el Capital Inicial: $ 349186.3866468654
RDTO: 0.3491863866468654
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 307522.55377455434
Dinero Final Débil: $ 910810.0740109651
Dinero Total Final: $ 1218332.6277855195
Diferencia con el Capital Inicial: $ 218332.62778551946
RDTO: 0.21833262778551954
Dinero inicial: $ 1000000
Dinero Final Fuerte: $ 306766.31861755694
Dinero Final Débil: $ 100

IndexError: index 200 is out of bounds for axis 0 with size 199

### Max Drawdown


In [3]:
#Se extraen los rendimientos semanales por año
rdtos = pd.read_excel('Rdtos.xlsx')

Función que calcula el max drawdown

In [4]:
def calcular_max_drawdown(valores_inversion):
    # Inicializar variables
    pico_actual = 0
    valle_actual = 0
    max_drawdown = 0

    for valor_actual in valores_inversion:
        if valor_actual > pico_actual:
            pico_actual = valor_actual
            valle_actual = valor_actual
        else:
            drawdown_actual = (pico_actual - valor_actual) / pico_actual
            max_drawdown = max(drawdown_actual, max_drawdown)

    return max_drawdown

In [5]:
# Inicializar un DataFrame para almacenar los resultados
resultados_max_drawdown = pd.DataFrame(columns=['Año', 'Max_Drawdown'])

# Iterar sobre las columnas del DataFrame
for nombre_columna, columna in rdtos.items():
    # Aplicar dropna() a la columna y calcular el máximo drawdown
    drawdown_actual = calcular_max_drawdown(columna.dropna())

    # Almacenar el resultado en el DataFrame de resultados
    resultados_max_drawdown = pd.concat([resultados_max_drawdown, pd.DataFrame({'Año': [nombre_columna], 'Max_Drawdown': [drawdown_actual]})], ignore_index=True)

# Encontrar el máximo drawdown entre todos los años
max_drawdown_total = resultados_max_drawdown['Max_Drawdown'].max()

# Imprimir el resultado
print(f'Mayor Drawdown: {max_drawdown_total:.2%}')

# Imprimir resultados
print("Max drawdown por año:")
print(resultados_max_drawdown)

Mayor Drawdown: 100.08%
Max drawdown por año:
    Año  Max_Drawdown
0  2015      0.874393
1  2016      0.822482
2  2017      0.784820
3  2018      0.973731
4  2020      0.931808
5  2021      0.922448
6  2022      1.000771
7  2023      0.735992


C:\Users\diwed\AppData\Local\Temp\ipykernel_28744\2524327636.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resultados_max_drawdown = pd.concat([resultados_max_drawdown, pd.DataFrame({'Año': [nombre_columna], 'Max_Drawdown': [drawdown_actual]})], ignore_index=True)


In [7]:
#Var y CVaR
# Nivel de confianza
confianza = 0.95

# Inicializar un DataFrame para almacenar los resultados de Var y CVaR
resultados_var_cvar = pd.DataFrame(columns=['Año', 'Var', 'CVaR'])

# Iterar sobre las columnas del DataFrame
for nombre_columna, columna in rdtos.items():
    # Calcular la Var
    var_actual = np.percentile(columna.dropna(), 100 * (1 - confianza))

    # Calcular la CVaR
    cvar_actual = columna[columna <= var_actual].mean()

    # Almacenar el resultado en el DataFrame de resultados
    resultados_var_cvar = pd.concat([resultados_var_cvar, pd.DataFrame({'Año': [nombre_columna], 'Var': [var_actual], 'CVaR': [cvar_actual]})], ignore_index=True)

# Desactivar la notación científica en pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Imprimir resultados
print(resultados_var_cvar)

    Año        Var       CVaR
0  2015  215417.01  193807.10
1  2016  749242.46  608317.61
2  2017 1106722.08 1039709.76
3  2018  103243.34   89784.06
4  2020  350309.72  324811.24
5  2021  184594.59  153695.82
6  2022    -904.11   -1473.70
7  2023 1113095.06 1045969.99


C:\Users\diwed\AppData\Local\Temp\ipykernel_28744\1740610376.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resultados_var_cvar = pd.concat([resultados_var_cvar, pd.DataFrame({'Año': [nombre_columna], 'Var': [var_actual], 'CVaR': [cvar_actual]})], ignore_index=True)
